In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import src.Fluid_CP as FCP
import src.Compressormodel_SP as CSP
from scipy.integrate import odeint
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

In [87]:
# Constants and parameters (as you have)
tc_s = 12*60          # total cooking time seconds
tb_s = 5*60          # pasta added after 5 min
R_env = 30 / 1000    # K/W
R_pasta = 15 / 1000  # K/W
c_l = 4.18 * 1000    # J/kg·K water
c_pasta = 3.5 * 1000 # J/kg·K pasta
Q_hp_W = 3984.26 # W

V_water = (47-0.4)*(27-0.4)*(20-0.2)/1e6  # m3
m_water = V_water * 1000                   # kg
m_pasta = 2                              # kg

T_env = 20
T_water_0 = 85
T_pasta_0 = 7

dt = 1  # 1 second timestep
time_steps = int(tc_s/dt) + 1
t = np.linspace(0, tc_s, time_steps)


T_WP_on = 85 #°C
T_WP_off = 93 #°C
rho_water=997. #kg/m3
WP = 1
time = 0

In [88]:
def calc_WP(T,T_WP_on,T_WP_off,op_times):
    global WP,time
    if WP==0 and T>T_WP_on:
        Q_WP=0. #kW WP off
        WP=0
        #time=0
    if WP==0 and T<=T_WP_on:
        Q_WP=4000 #kW WP switched on
        WP=1 #1=WP on
        time+=1
    if WP==1 and T<=T_WP_off:
        Q_WP=4000 #kW WP on
        time+=1
    if WP==1 and T>T_WP_off:
        Q_WP=0. #kW WP switched off
        WP=0
        op_times.append(time)
        time=0
    return Q_WP

### Keeping track of the temperatures

In [97]:
def temp_store_only_water(T_water,t,Q_WP,m_store,c_water):
    dT_waterdt = (Q_WP - (T_water - T_env)/R_env) / (m_store * c_water)
    return dT_waterdt


def temp_store_with_pasta(T, t, Q_WP, m_store, m_pasta, c_water):
    T_water, T_pasta = T  # unpack states
    dT_waterdt = (Q_WP - (T_water - T_env)/R_env - (T_water - T_pasta)/R_pasta) / (m_store * c_water)
    dT_pastadt = ((T_water - T_pasta) / R_pasta) / (m_pasta * c_pasta)
    return [dT_waterdt, dT_pastadt]

### Create binary list for when pasta is in water

In [98]:
def make_pasta_list(t):
    pasta_counter = 7*60 #seconds with pasta
    no_pasta_counter = 5*60  #seconds without pasta
    pasta_presence = np.zeros(len(t))
    
    for i in range(len(t)):
        if pasta_counter > 0:
            pasta_presence[i] = 1
            pasta_counter -= 1
        elif no_pasta_counter > 0:
            pasta_presence[i] = 0
            no_pasta_counter -= 1
        else:
            pasta_counter = 7*60-1
            no_pasta_counter = 5*60
            pasta_presence[i] = 1
            
        
    return pasta_presence



### Operation

In [101]:
def T_store_and_pasta(T_WP_on,T_WP_off,V_store,n,op_times):
    
    #initial storage temperature
    T0_water = T_WP_on #fully charged
    T0_pasta = 7
    # time points
    t = np.linspace(0,n,n)
    # store storage and pasta temperature
    T_water=np.zeros(n)
    T_pasta = np.full(n, 7)
    # save initial storage and pasta temperature
    T_water[0] = T0_water
    T_pasta[0] = 7
    # mass water in storage
    m_store=V_store/1e3*rho_water #kg
    c_water = 4.18*1000 #J/kgK
    m_pasta = 2 #kg
    c_pasta = 3.5*1000 #J/kgK
    
    # heat pump
    Q_WPs=np.zeros(n) #store power of heat pump
    Q_WPs[0]=0 #kW power of heat pump at t=0 is 0 kW
    
    #make binary list for when pasta is in water -> first 7min in water
    binary_pasta = make_pasta_list(t)

    # solve differential equation for the storage temperature
    for i in range(1,n):
        if binary_pasta[i] == 0:
            # span for next time step
            tspan = [t[i-1],t[i]]
            # solve for next step
            z = odeint(temp_store_only_water,T0_water,tspan,args=(Q_WPs[i-1],m_store,c_water))
            #heat pump power for netxt time step
            Q_WPs[i]=calc_WP(z[1],T_WP_on,T_WP_off,op_times)
            # store storage and pasta temperature
            T_water[i] = z[1]
            T_pasta[i] = 7
            # next initial condition
            T0_water = z[1]
            #reset temperature for T_pasta
            T0_pasta = 7 #°C
            
        else:
            tspan = np.linspace(t[i-1], t[i], num=10)  # finer steps between two main time steps
            T_function = [T0_water, T0_pasta]

            sol = odeint(temp_store_with_pasta, T_function, tspan, args=(Q_WPs[i-1], m_store, m_pasta, c_water))
            
            # update for next iteration
            T0_water = sol[-1, 0]
            T0_pasta = sol[-1, 1]
            print(T0_water)
            print(T0_pasta)

            # calculate heat pump power for next step using last water temp only
            Q_WPs[i] = calc_WP(T0_water, T_WP_on, T_WP_off, op_times)

            T_water[i] = T0_water
            T_pasta[i] = T0_pasta


            
            
    # plot results    
    #fig, ax1 = plt.subplots()
    #ax2 = ax1.twinx()
    #ax1.plot(t,T,'r--',label='T_store')
    #ax2.plot(t,Q_WPs,'g:',label='Q_WP')
    #ax1.set_xlabel('time in seconds')
    #ax1.set_ylabel('T_store in °C', color='r')
    #ax2.set_ylabel('Q_WP in kW', color='g')
    #plt.show()
    
    # plot results    
    fig, (ax1,ax2) = plt.subplots(2,sharex=True)

    ax1.plot(t,T_water,'r--',label='T_store')
    ax2.plot(t,Q_WPs,'g:',label='Q_WP')
    ax2.set_xlabel('time in seconds')
    ax1.set_ylabel('T_store in °C', color='r')
    ax2.set_ylabel('Q_WP in kW', color='g')
    plt.show()
    
    
    return t,Q_WPs,T_water,T_pasta

In [102]:
V_store = (47-0.4) * (27-0.4) * (20-0.2) /1000 #L
n = 12*60*60 #sec
op_times=list() #list for operating times

#calculation temperature profile and heat pump operating times
t1,Q_WPs1,T_water,T_pasta=T_store_and_pasta(T_WP_on,T_WP_off,V_store,n,op_times)

#if heat pump is running at the end of the time range, the running time is added to the list
if time!=0:
    op_times.append(time)
print("Min operating time=",round(min(op_times)/60.,2),"minutes")
print("Max operating time=",round(max(op_times)/60.,2),"minutes")
print("Number of times heat pump was switched on=",len(op_times))

84.9282524505242
7.739006777605999
84.89614288720543
8.470517052302503
84.864538905592
9.194791526992812
84.83343532160019
9.911903555685292
84.80282700374548
10.62192574837106
84.77270887261795
11.324929978442965
84.74307590035575
12.020987390144994
84.71392311012278
12.710168405958257
84.68524557559118
13.392542733923214
84.65703842042953
14.06817937487749
84.62929681779539
14.737146629635275
84.60201598983342
15.399512106085387
84.5751912071786
16.05534272621733
84.54881779345733
16.704704662429183
84.52289110914204
17.347663565983513
84.49740656743496
17.984284342248557
84.47235962809182
18.614631238089867
84.44774579694979
19.238767848548136
84.42356062545913
19.856757123463815
84.3997997102205
20.468661374021696
84.37645869252653
21.074542279236116
84.35353325790831
21.674460892363832
84.33101913568642
22.268477647257562
84.30891209852658
22.856652364648827
84.2872079619996
23.439044258373357
84.26590258414613
24.015711941527805
84.24499186504576
24.58671343256544
84.224471746390

/var/folders/zr/7bf2xpyx49d4nx1_77v7ygqm0000gn/T/ipykernel_17292/1606374334.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  T_water[i] = z[1]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.